## 1. Setup

In [1]:
import sys
import os
import shutil
import h5py
from glob import glob
import pathlib
from pathlib import Path
from collections import defaultdict
from copy import deepcopy
from tqdm import tqdm
import yaml

import numpy as np
import pandas as pd
from PIL import Image, ExifTags
import cv2
import sqlite3

from sklearn.metrics.pairwise import cosine_similarity

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, SequentialSampler
import timm
import kornia
from kornia.feature import LoFTR

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import pycolmap

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
competition_dataset = Path('/kaggle/input/image-matching-challenge-2023')
external_dataset = Path('/kaggle/input/image-matching-challenge-2023-dataset')

In [3]:
sys.path.append(str(external_dataset / 'packages' / 'SuperGluePretrainedNetwork'))
from models.matching import Matching

In [4]:
df = pd.read_csv(competition_dataset / 'sample_submission.csv')

if df.shape[0] != 3:
    # Enable submission mode and disable verbose
    submission = True
    verbose = False
    train_or_test_directory = competition_dataset / 'test'
else:
    # Disable submission mode and enable verbose
    submission = False
    verbose = True
    train_or_test_directory = competition_dataset / 'train'

if submission is False:
    df = pd.read_csv(competition_dataset / 'train' / 'train_labels.csv')
    # Use only bike scene from the haiper dataset if it's not submission mode
    df = df.loc[df['scene'] == 'bike'].reset_index(drop=True)
    
# Extract image id, image height and image width from images
for idx, row in tqdm(df.iterrows(), total=df.shape[0]):

    image_path = train_or_test_directory / row['image_path']
    image = cv2.imread(str(image_path))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    df.loc[idx, 'image_height'] = image.shape[0]
    df.loc[idx, 'image_width'] = image.shape[1]


df['image_id'] = df['image_path'].apply(lambda x: str(x).split('/')[-1])

100%|██████████| 15/15 [00:00<00:00, 17.87it/s]


In [5]:
df

,dataset,scene,image_path,rotation_matrix,translation_vector,image_height,image_width,image_id
0,haiper,bike,haiper/bike/images/image_004.jpeg,-0.0444086367008516;-0.9943622104926982;0.0962...,-0.1960672197513899;-1.7201514631645476;2.9038...,1920.0,1440.0,image_004.jpeg
1,haiper,bike,haiper/bike/images/image_029.jpeg,-0.30379429789458934;-0.780428110344117;0.5464...,-0.7743795049636117;-2.4063622468195978;4.2039...,1920.0,1440.0,image_029.jpeg
2,haiper,bike,haiper/bike/images/image_038.jpeg,-0.44075875074949056;-0.6141344795467536;0.654...,-0.8061619977689489;-1.3614914099692277;2.5759...,1920.0,1440.0,image_038.jpeg
3,haiper,bike,haiper/bike/images/image_049.jpeg,-0.4615070783507189;-0.037215312681071866;0.88...,-0.05029237007998646;-2.0953351075544067;3.128...,1920.0,1440.0,image_049.jpeg
4,haiper,bike,haiper/bike/images/image_062.jpeg,0.06454466684801141;0.9943063402602297;0.08478...,0.45813698932578173;-1.7352377000311676;3.6787...,1920.0,1440.0,image_062.jpeg
5,haiper,bike,haiper/bike/images/image_076.jpeg,0.566876909470875;-0.09510152245268588;-0.8182...,1.1496049380742543;-1.4930263438651057;2.57378...,1920.0,1440.0,image_076.jpeg
6,haiper,bike,haiper/bike/images/image_088.jpeg,0.253507028584536;-0.8760074072730466;-0.41029...,-0.44871563031823725;-1.750899302580256;2.6802...,1920.0,1440.0,image_088.jpeg
7,haiper,bike,haiper/bike/images/image_094.jpeg,0.15438718127182738;-0.9293944893965761;-0.335...,-0.37595896679394963;-2.714322519243274;4.1322...,1920.0,1440.0,image_094.jpeg
8,haiper,bike,haiper/bike/images/image_101.jpeg,0.4234658773801474;-0.6722258122106511;-0.6072...,-0.11276896793699917;-0.7117396520115244;1.914...,1920.0,1440.0,image_101.jpeg
9,haiper,bike,haiper/bike/images/image_115.jpeg,0.3904043657227909;0.16367189264027227;-0.9059...,0.8296975499797659;-2.3480417943432315;3.47250...,1920.0,1440.0,image_115.jpeg


## 2. Image Utilities

In [6]:
def create_image_pairs(image_paths):

    """
    Create all possible image pairs from given list of image paths

    Parameters
    ----------
    image_paths: list of shape (n_images)
        List of image paths

    Returns
    -------
    image_pair_indices: list of shape (n_image_pairs)
        List of tuples of image pair indices
    """

    image_pair_indices = []

    for i in range(len(image_paths)):
        for j in range(i + 1, len(image_paths)):
            image_pair_indices.append((i, j))

    return image_pair_indices


def resize_with_aspect_ratio(image, longest_edge):

    """
    Resize image while preserving its aspect ratio

    Parameters
    ----------
    image: numpy.ndarray of shape (height, width, 3)
        Image array

    longest_edge: int
        Desired number of pixels on the longest edge

    Returns
    -------
    image: numpy.ndarray of shape (resized_height, resized_width, 3)
        Resized image array
    """

    height, width = image.shape[:2]
    scale = longest_edge / max(height, width)
    image = cv2.resize(image, dsize=(int(width * scale), int(height * scale)), interpolation=cv2.INTER_LANCZOS4)

    return image


def get_image_tensor(image_path_or_array, resize, resize_shape, resize_longest_edge, scale, grayscale):

    """
    Load image and preprocess it

    Parameters
    ----------
    image_path_or_array: str or numpy.ndarray of shape (height, width, 3)
        Image path or image array

    resize: bool
        Whether to resize the image or not

    resize_shape: tuple or int
        Tuple of image height and width or number of pixels for both height and width

    resize_longest_edge: bool
        Whether to resize the longest edge or not

    scale: bool
        Whether to scale image pixel values by max 8-bit pixel value or not

    grayscale: bool
        Whether to convert RGB image to grayscale or not

    Returns
    -------
    image: torch.Tensor of shape (1, 1 or 3, height, width)
        Image tensor
    """

    if isinstance(image_path_or_array, pathlib.Path) or isinstance(image_path_or_array, str):
        # Read image from the given path if image_path_or_array is a path-like string
        image = cv2.imread(str(image_path_or_array))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    else:
        image = image_path_or_array

    if resize:
        if resize_longest_edge:
            image = resize_with_aspect_ratio(image=image, longest_edge=resize_shape)
        else:
            resize_shape = (resize_shape, resize_shape) if isinstance(resize_shape, int) else resize_shape
            image = cv2.resize(image, resize_shape, interpolation=cv2.INTER_LANCZOS4)

    if scale:
        image = image / 255.

    image = kornia.image_to_tensor(image, False).float()
    if grayscale:
        image = kornia.color.rgb_to_grayscale(image)

    return image


## 3. Camera Utilities

In [7]:
def get_focal_length(image_path):

    """
    Get focal length from EXIF or calculate it using prior

    Parameters
    ----------
    image_path: str
        Image path

    Returns
    -------
    focal_length: float
        Focal length extracted from EXIF or calculated using prior
    """

    image = Image.open(image_path)
    image_longest_edge = max(image.size)

    focal_length = None
    exif = image.getexif()

    if exif is not None:

        focal_length_35mm = None

        for tag, value in exif.items():
            if ExifTags.TAGS.get(tag, None) == 'FocalLengthIn35mmFilm':
                focal_length_35mm = float(value)

        if focal_length_35mm is not None:
            focal_length = focal_length_35mm / 35. * image_longest_edge

    if focal_length is None:
        prior_focal_length = 1.2
        focal_length = prior_focal_length * image_longest_edge

    return focal_length


## 4. Database Utilities

In [8]:
MAX_IMAGE_ID = 2 ** 31 - 1

CREATE_CAMERAS_TABLE_QUERY = '''
CREATE TABLE IF NOT EXISTS cameras (
    camera_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    model INTEGER NOT NULL,
    width INTEGER NOT NULL,
    height INTEGER NOT NULL,
    params BLOB,
    prior_focal_length INTEGER NOT NULL
)
'''

CREATE_IMAGES_TABLE_QUERY = f'''
CREATE TABLE IF NOT EXISTS images (
    image_id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL,
    name TEXT NOT NULL UNIQUE,
    camera_id INTEGER NOT NULL,
    prior_qw REAL,
    prior_qx REAL,
    prior_qy REAL,
    prior_qz REAL,
    prior_tx REAL,
    prior_ty REAL,
    prior_tz REAL,
    CONSTRAINT image_id_check CHECK(image_id >= 0 and image_id < {MAX_IMAGE_ID}),
    FOREIGN KEY(camera_id) REFERENCES cameras(camera_id)
)
'''

CREATE_DESCRIPTORS_TABLE_QUERY = '''
CREATE TABLE IF NOT EXISTS descriptors (
    image_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB,
    FOREIGN KEY(image_id) REFERENCES images(image_id) ON DELETE CASCADE
)
'''

CREATE_KEYPOINTS_TABLE_QUERY = '''CREATE TABLE IF NOT EXISTS keypoints (
    image_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB,
    FOREIGN KEY(image_id) REFERENCES images(image_id) ON DELETE CASCADE
)
'''

CREATE_MATCHES_TABLE_QUERY = '''
CREATE TABLE IF NOT EXISTS matches (
    pair_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB
)
'''

CREATE_TWO_VIEW_GEOMETRIES_TABLE_QUERY = '''
CREATE TABLE IF NOT EXISTS two_view_geometries (
    pair_id INTEGER PRIMARY KEY NOT NULL,
    rows INTEGER NOT NULL,
    cols INTEGER NOT NULL,
    data BLOB,
    config INTEGER NOT NULL,
    F BLOB,
    E BLOB,
    H BLOB
)
'''

CREATE_NAME_INDEX_QUERY = 'CREATE UNIQUE INDEX IF NOT EXISTS index_name ON images(name)'

CREATE_ALL_QUERY = '; '.join([
    CREATE_CAMERAS_TABLE_QUERY,
    CREATE_IMAGES_TABLE_QUERY,
    CREATE_KEYPOINTS_TABLE_QUERY,
    CREATE_DESCRIPTORS_TABLE_QUERY,
    CREATE_MATCHES_TABLE_QUERY,
    CREATE_TWO_VIEW_GEOMETRIES_TABLE_QUERY,
    CREATE_NAME_INDEX_QUERY
])


def blob_to_array(x):
    if x is not None:
        return np.frombuffer(x)


def array_to_blob(x):
    return np.asarray(x).tobytes()


def image_ids_to_pair_id(image1_id, image2_id):
    if image1_id > image2_id:
        image1_id, image2_id = image2_id, image1_id
    return image1_id * MAX_IMAGE_ID + image2_id


def pair_id_to_image_ids(pair_id):
    image_id2 = pair_id % MAX_IMAGE_ID
    image_id1 = (pair_id - image_id2) / MAX_IMAGE_ID
    return image_id1, image_id2


class COLMAPDatabase(sqlite3.Connection):

    @staticmethod
    def connect(database_path, uri):
        return sqlite3.connect(database_path, uri=uri, factory=COLMAPDatabase)

    def __init__(self, *args, **kwargs):

        super(COLMAPDatabase, self).__init__(*args, **kwargs)

        self.create_tables = lambda: self.executescript(CREATE_ALL_QUERY)
        self.create_cameras_table = lambda: self.executescript(CREATE_CAMERAS_TABLE_QUERY)
        self.create_descriptors_table = lambda: self.executescript(CREATE_DESCRIPTORS_TABLE_QUERY)
        self.create_images_table = lambda: self.executescript(CREATE_IMAGES_TABLE_QUERY)
        self.create_two_view_geometries_table = lambda: self.executescript(CREATE_TWO_VIEW_GEOMETRIES_TABLE_QUERY)
        self.create_keypoints_table = lambda: self.executescript(CREATE_KEYPOINTS_TABLE_QUERY)
        self.create_matches_table = lambda: self.executescript(CREATE_MATCHES_TABLE_QUERY)
        self.create_name_index = lambda: self.executescript(CREATE_NAME_INDEX_QUERY)

    def get_cameras_table(self):

        df_cameras = pd.read_sql('SELECT * FROM cameras', con=self)
        if df_cameras.shape[0] > 0:
            df_cameras['params'] = df_cameras['params'].apply(lambda x: blob_to_array(x))

        return df_cameras

    def get_images_table(self):

        df_images = pd.read_sql('SELECT * FROM images', con=self)

        return df_images

    def get_descriptors_table(self):

        df_descriptors = pd.read_sql('SELECT * FROM descriptors', con=self)
        if df_descriptors.shape[0] > 0:
            df_descriptors['data'] = df_descriptors['data'].apply(lambda x: blob_to_array(x))

        return df_descriptors

    def get_keypoints_table(self):

        df_keypoints = pd.read_sql('SELECT * FROM keypoints', con=self)
        if df_keypoints.shape[0] > 0:
            df_keypoints['data'] = df_keypoints['data'].apply(lambda x: blob_to_array(x))

        return df_keypoints

    def get_matches_table(self):

        df_matches = pd.read_sql('SELECT * FROM matches', con=self)
        if df_matches.shape[0] > 0:
            df_matches['data'] = df_matches['data'].apply(lambda x: blob_to_array(x))

        return df_matches

    def get_two_view_geometries_table(self):

        df_two_view_geometries = pd.read_sql('SELECT * FROM two_view_geometries', con=self)
        if df_two_view_geometries.shape[0] > 0:
            df_two_view_geometries['data'] = df_two_view_geometries['data'].apply(lambda x: blob_to_array(x))
            df_two_view_geometries['F'] = df_two_view_geometries['F'].apply(lambda x: blob_to_array(x))
            df_two_view_geometries['E'] = df_two_view_geometries['E'].apply(lambda x: blob_to_array(x))
            df_two_view_geometries['H'] = df_two_view_geometries['H'].apply(lambda x: blob_to_array(x))
            df_two_view_geometries['qvec'] = df_two_view_geometries['qvec'].apply(lambda x: blob_to_array(x))
            df_two_view_geometries['tvec'] = df_two_view_geometries['tvec'].apply(lambda x: blob_to_array(x))

        return df_two_view_geometries

    def add_camera(self, camera_id, model, width, height, params, prior_focal_length):

        cursor = self.execute(
            'INSERT INTO cameras VALUES (?, ?, ?, ?, ?, ?)',
            (
                camera_id,
                model,
                width,
                height,
                array_to_blob(params),
                prior_focal_length
            )
        )

        return cursor.lastrowid

    def add_image(self, image_id, name, camera_id, prior_q=np.zeros(4), prior_t=np.zeros(3)):

        cursor = self.execute(
            'INSERT INTO images VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)',
            (
                image_id,
                name,
                camera_id,
                prior_q[0],
                prior_q[1],
                prior_q[2],
                prior_q[3],
                prior_t[0],
                prior_t[1],
                prior_t[2]
            )
        )

        return cursor.lastrowid

    def add_descriptors(self, image_id, descriptors):

        descriptors = np.ascontiguousarray(descriptors, np.uint8)

        self.execute(
            'INSERT INTO descriptors VALUES (?, ?, ?, ?)',
            (
                    image_id,
                    descriptors.shape[0],
                    descriptors.shape[1],
                    array_to_blob(descriptors)
            )
        )

    def add_keypoints(self, image_id, keypoints):

        keypoints = np.asarray(keypoints, np.float32)

        self.execute(
            'INSERT INTO keypoints VALUES (?, ?, ?, ?)',
            (
                image_id,
                keypoints.shape[0],
                keypoints.shape[1],
                array_to_blob(keypoints)
            )
        )

    def add_matches(self, image1_id, image2_id, matches):

        if image1_id > image2_id:
            matches = matches[:, ::-1]

        pair_id = image_ids_to_pair_id(image1_id, image2_id)
        matches = np.asarray(matches, np.uint32)

        self.execute(
            'INSERT INTO matches VALUES (?, ?, ?, ?)',
            (
                pair_id,
                matches.shape[0],
                matches.shape[1],
                array_to_blob(matches)
            )
        )

    def add_two_view_geometries(self, image1_id, image2_id, matches, config=2, F=np.eye(3), E=np.eye(3), H=np.eye(3)):

        if image1_id > image2_id:
            matches = matches[:, ::-1]

        pair_id = image_ids_to_pair_id(image1_id, image2_id)
        matches = np.asarray(matches, np.uint32)

        F = np.asarray(F, dtype=np.float64)
        E = np.asarray(E, dtype=np.float64)
        H = np.asarray(H, dtype=np.float64)

        self.execute(
            'INSERT INTO two_view_geometries VALUES (?, ?, ?, ?, ?, ?, ?, ?)',
            (
                pair_id,
                matches.shape[0],
                matches.shape[1],
                array_to_blob(matches),
                config,
                array_to_blob(F),
                array_to_blob(E),
                array_to_blob(H)
            )
        )


def get_first_indices(x, dim=0):

    """
    Get indices where values appear first

    Parameters
    ----------
    x: torch.Tensor
        N-dimensional torch tensor

    dim: int
        Dimension of the unique operation

    Returns
    -------
    first_indices: torch.Tensor
        Indices where values appear first
    """

    _, idx, counts = torch.unique(x, dim=dim, sorted=True, return_inverse=True, return_counts=True)
    _, sorted_idx = torch.sort(idx, stable=True)
    counts_cumsum = counts.cumsum(0)
    counts_cumsum = torch.cat((torch.tensor([0], device=counts_cumsum.device), counts_cumsum[:-1]))
    first_indices = sorted_idx[counts_cumsum]

    return first_indices


def write_matches(image_paths, image_pair_indices, first_image_keypoints, second_image_keypoints, output_directory):

    """
    Write matches as h5 datasets for COLMAP

    Parameters
    ----------
    image_paths: list of shape (n_images)
        List of image paths

    image_pair_indices: list of shape (n_image_pairs)
        List of tuples of image pair indices

    first_image_keypoints: list of shape (n_image_pairs)
        List of first image keypoints

    second_image_keypoints: list of shape (n_image_pairs)
        List of second image keypoints

    output_directory: str or pathlib.Path object
        Path of the output directory
    """

    with h5py.File(output_directory / 'loftr_matches.h5', mode='w') as f:
        for matching_index, image_pair_index in enumerate(image_pair_indices):
            first_image_path, second_image_path = image_paths[image_pair_index[0]], image_paths[image_pair_index[1]]
            first_image_filename, second_image_filename = first_image_path.split('/')[-1], second_image_path.split('/')[-1]

            # Concatenate matched keypoints of an image pair and write it as a dataset
            group = f.require_group(first_image_filename)
            group.create_dataset(
                second_image_filename,
                data=np.concatenate([first_image_keypoints[matching_index], second_image_keypoints[matching_index]], axis=1)
            )

    keypoints = defaultdict(list)
    match_indices = defaultdict(dict)
    total_keypoints = defaultdict(int)

    with h5py.File(output_directory / 'loftr_matches.h5', mode='r') as f:
        for first_image_filename in f.keys():
            group = f[first_image_filename]
            for second_image_filename in group.keys():

                image_pair_keypoints = group[second_image_filename][...]
                keypoints[first_image_filename].append(image_pair_keypoints[:, :2])
                keypoints[second_image_filename].append(image_pair_keypoints[:, 2:])
                current_match = torch.arange(len(image_pair_keypoints)).reshape(-1, 1).repeat(1, 2)
                current_match[:, 0] += total_keypoints[first_image_filename]
                current_match[:, 1] += total_keypoints[second_image_filename]
                total_keypoints[first_image_filename] += len(image_pair_keypoints)
                total_keypoints[second_image_filename] += len(image_pair_keypoints)
                match_indices[first_image_filename][second_image_filename] = current_match

    for image_filename in keypoints.keys():
        keypoints[image_filename] = np.round(np.concatenate(keypoints[image_filename], axis=0))

    unique_keypoints = {}
    unique_match_indices = {}
    matches = defaultdict(dict)

    for image_filename in keypoints.keys():
        unique_keypoint_values, unique_keypoint_reverse_idx = torch.unique(torch.from_numpy(keypoints[image_filename]), dim=0, return_inverse=True)
        unique_match_indices[image_filename] = unique_keypoint_reverse_idx
        unique_keypoints[image_filename] = unique_keypoint_values.numpy()

    for first_image_filename, group in match_indices.items():
        for second_image_filename, image_pair_match_index in group.items():
            image_pair_match_index_copy = deepcopy(image_pair_match_index)
            image_pair_match_index_copy[:, 0] = unique_match_indices[first_image_filename][image_pair_match_index_copy[:, 0]]
            image_pair_match_index_copy[:, 1] = unique_match_indices[second_image_filename][image_pair_match_index_copy[:, 1]]
            matched_keypoints = np.concatenate([
                unique_keypoints[first_image_filename][image_pair_match_index_copy[:, 0]].reshape(-1, 2),
                unique_keypoints[second_image_filename][image_pair_match_index_copy[:, 1]].reshape(-1, 2)
            ], axis=1)

            if matched_keypoints.shape[0] == 0:
                continue

            current_unique_match_index = get_first_indices(torch.from_numpy(matched_keypoints), dim=0)
            image_pair_match_index_copy_semiclean = image_pair_match_index_copy[current_unique_match_index]

            current_unique_match_index1 = get_first_indices(image_pair_match_index_copy_semiclean[:, 0], dim=0)
            image_pair_match_index_copy_semiclean = image_pair_match_index_copy_semiclean[current_unique_match_index1]

            current_unique_match_index2 = get_first_indices(image_pair_match_index_copy_semiclean[:, 1], dim=0)
            image_pair_match_index_copy_semiclean2 = image_pair_match_index_copy_semiclean[current_unique_match_index2]

            matches[first_image_filename][second_image_filename] = image_pair_match_index_copy_semiclean2.numpy()

    with h5py.File(output_directory / 'keypoints.h5', mode='w') as f:
        for image_filename, keypoints in unique_keypoints.items():
            f[image_filename] = keypoints

    with h5py.File(output_directory / 'matches.h5', mode='w') as f:
        for first_image_filename, first_image_matches in matches.items():
            group = f.require_group(first_image_filename)
            for second_image_filename, second_image_matches in first_image_matches.items():
                group[second_image_filename] = second_image_matches


def push_to_database(colmap_database, dataset_directory, image_directory, camera_model, single_camera):

    """
    Push cameras, images, keypoints and matches to COLMAP database

    Parameters
    ----------
    colmap_database: COLMAPDatabase
        COLMAP Database object

    dataset_directory: str or pathlib.Path
        Reconstruction directory

    image_directory: str or pathlib.Path
        Image directory

    camera_model: str
        Model of the camera

    single_camera: bool
        Whether there is one or multiple in cameras
    """

    keypoints_dataset = h5py.File(dataset_directory / 'keypoints.h5', 'r')

    camera_id = None
    image_filename_to_id = {}

    for image_filename in tqdm(list(keypoints_dataset.keys())):

        keypoints = keypoints_dataset[image_filename][()]

        if camera_id is None or not single_camera:

            image = Image.open(str(image_directory / image_filename))
            width, height = image.size

            focal_length = get_focal_length(image_path=str(image_directory / image_filename))

            if camera_model == 'simple-pinhole':
                model = 0
                params = np.array([focal_length, width / 2, height / 2])
            elif camera_model == 'pinhole':
                model = 1
                params = np.array([focal_length, focal_length, width / 2, height / 2])
            elif camera_model == 'simple-radial':
                model = 2
                params = np.array([focal_length, width / 2, height / 2, 0.1])
            elif camera_model == 'opencv':
                model = 4
                params = np.array([focal_length, focal_length, width / 2, height / 2, 0., 0., 0., 0.])
            else:
                raise ValueError(f'Invalid camera model: {camera_model}')

            camera_id = colmap_database.add_camera(
                camera_id=None,
                model=model,
                width=width,
                height=height,
                params=params,
                prior_focal_length=False
            )

        image_id = colmap_database.add_image(image_id=None, name=image_filename, camera_id=camera_id)
        image_filename_to_id[image_filename] = image_id

        colmap_database.add_keypoints(image_id=image_id, keypoints=keypoints)

    matches_dataset = h5py.File(dataset_directory / 'matches.h5', 'r')

    pairs = set()

    for image1_filename in matches_dataset.keys():
        group = matches_dataset[image1_filename]
        for image2_filename in group.keys():

            image1_id = image_filename_to_id[image1_filename]
            image2_id = image_filename_to_id[image2_filename]
            pair_id = image_ids_to_pair_id(image1_id=image1_id, image2_id=image2_id)
            if pair_id in pairs:
                continue

            matches = group[image2_filename][()]
            colmap_database.add_matches(image1_id, image2_id, matches)
            pairs.add(pair_id)

    colmap_database.commit()


## 5. Datasets

In [9]:
class ImageDataset(Dataset):

    def __init__(self, image_paths, transforms=None):

        self.image_paths = image_paths
        self.transforms = transforms

    def __len__(self):

        """
        Get the length the dataset

        Returns
        -------
        length: int
            Length of the dataset
        """

        return len(self.image_paths)

    def __getitem__(self, idx):

        """
        Get the idxth element in the dataset

        Parameters
        ----------
        idx: int
            Index of the sample (0 <= idx < length of the dataset)

        Returns
        -------
        image: torch.FloatTensor of shape (channel, height, width) or numpy.ndarray of shape (height, width, channel)
            Image tensor or array
        """

        image = cv2.imread(str(self.image_paths[idx]))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        if self.transforms is not None:
            image = self.transforms(image=np.array(image))['image'].float()

        return image


class ImagePairDataset(Dataset):

    def __init__(self, image_paths, image_pair_indices, transforms):

        self.image_paths = image_paths
        self.image_pair_indices = image_pair_indices
        self.transforms = transforms

    def __len__(self):

        """
        Get the length the dataset

        Returns
        -------
        length: int
            Length of the dataset
        """

        return len(self.image_pair_indices)

    def __getitem__(self, idx):

        """
        Get the idxth element in the dataset

        Parameters
        ----------
        idx: int
            Index of the sample (0 <= idx < length of the dataset)

        Returns
        -------
        image1: torch.FloatTensor of shape (channel, height, width)
            First image tensor

        image2: torch.FloatTensor of shape (channel, height, width)
            Second image tensor
        """

        image1 = get_image_tensor(
            image_path_or_array=str(self.image_paths[self.image_pair_indices[idx][0]]),
            resize_shape=self.transforms['resize_shape'],
            resize_longest_edge=self.transforms['resize_longest_edge'],
            scale=self.transforms['scale'],
            grayscale=self.transforms['grayscale']
        )

        image2 = get_image_tensor(
            image_path_or_array=str(self.image_paths[self.image_pair_indices[idx][1]]),
            resize_shape=self.transforms['resize_shape'],
            resize_longest_edge=self.transforms['resize_longest_edge'],
            scale=self.transforms['scale'],
            grayscale=self.transforms['grayscale']
        )

        return image1, image2


## 6. Image Selection

In [10]:
def load_feature_extractor(model_name, pretrained, model_args):

    """
    Load specified pretrained model for feature extraction

    Parameters
    ----------
    model_name: str
        Model name

    pretrained: bool
        Whether to load pretrained weights or not

    model_args: dict
        Dictionary of model keyword arguments

    Returns
    -------
    model: torch.nn.Module
        Model
    """

    model = timm.create_model(
        model_name=model_name,
        pretrained=pretrained,
        **model_args
    )
    # Override the head layer with Identity in transformer models so the output will be features
    model.head = nn.Identity()

    return model


def extract_features(inputs, model, pooling_type, device, amp):

    """
    Extract features from given inputs with given model

    Parameters
    ----------
    inputs: torch.FloatTensor of shape (batch, channel, height, width)
        Inputs tensor

    model: torch.nn.Module
        Model

    pooling_type: str
        Pooling type applied to features

    device: torch.device
        Location of the inputs tensor and the model

    amp: bool
        Whether to use auto mixed precision or not

    Returns
    -------
    features: torch.FloatTensor of shape (batch, features)
        Features tensor
    """

    with torch.no_grad():
        if amp:
            with torch.autocast(device_type=device.type, dtype=torch.bfloat16):
                if pooling_type is not None:
                    # Use forward features if pooling type is specified (convolutional models)
                    features = model.forward_features(inputs)
                else:
                    # Use forward if pooling type is not specified (transformer models)
                    features = model(inputs)
        else:
            if pooling_type is not None:
                features = model.forward_features(inputs)
            else:
                features = model(inputs)

    features = features.detach().cpu()

    if pooling_type == 'avg':
        features = F.adaptive_avg_pool2d(features, output_size=(1, 1)).view(features.size(0), -1)
    elif pooling_type == 'max':
        features = F.adaptive_max_pool2d(features, output_size=(1, 1)).view(features.size(0), -1)
    elif pooling_type == 'concat':
        features = torch.cat([
            F.adaptive_avg_pool2d(features, output_size=(1, 1)).view(features.size(0), -1),
            F.adaptive_max_pool2d(features, output_size=(1, 1)).view(features.size(0), -1)
        ], dim=-1)

    features = features / features.norm(dim=1).view(-1, 1)

    return features


def select_images(image_paths, image_selection_features, image_count):

    """
    Select most similar images

    Parameters
    ----------
    image_paths: list of shape (n_images)
        List of image paths

    image_selection_features: np.ndarray of shape (n_images, n_features)
        Features array

    image_count: int
        Image count to retrieve most similar images

    Returns
    -------
    image_paths: list of shape (image_count)
        List of most similar image paths
    """

    # Calculate pairwise cosine similarities between features
    pairwise_cosine_similarities = cosine_similarity(image_selection_features)

    # Zero the diagonal and calculate mean cosine similarities
    np.fill_diagonal(pairwise_cosine_similarities, 0)
    mean_cosine_similarities = pairwise_cosine_similarities.mean(axis=1)

    # Extract sorting index in descending order
    sorting_idx = np.argsort(mean_cosine_similarities)[::-1]

    image_paths = np.array(image_paths)
    image_paths = image_paths[sorting_idx][:image_count].tolist()

    return image_paths


def prepare_dataloader(image_paths, transforms, batch_size, num_workers):

    """
    Prepare data loader for inference

    Parameters
    ----------
    image_paths: list of shape (n_images)
        List of image paths

    transforms: dict
        Transform pipeline

    batch_size: int
        Batch size of the data loader

    num_workers: int
        Number of workers of the data loader

    Returns
    -------
    data_loader: torch.utils.data.DataLoader
        Data loader
    """

    dataset = ImageDataset(image_paths=image_paths, transforms=transforms)
    data_loader = DataLoader(
        dataset,
        batch_size=batch_size,
        sampler=SequentialSampler(dataset),
        pin_memory=False,
        drop_last=False,
        num_workers=num_workers
    )

    return data_loader


def create_image_selection_transforms(**transform_parameters):

    """
    Create transformation pipeline for image selection

    Parameters
    ----------
    transform_parameters: dict
        Dictionary of transform parameters

    Returns
    -------
    transforms: dict
        Transform pipeline for image selection
    """

    image_selection_transforms = A.Compose([
        A.Resize(
            height=transform_parameters['resize_height'],
            width=transform_parameters['resize_width'],
            interpolation=cv2.INTER_NEAREST,
            always_apply=True
        ),
        A.Normalize(
            mean=transform_parameters['normalize_mean'],
            std=transform_parameters['normalize_std'],
            max_pixel_value=transform_parameters['normalize_max_pixel_value'],
            always_apply=True
        ),
        ToTensorV2(always_apply=True)
    ])

    return image_selection_transforms


## 7. LoFTR

In [11]:
def loftr_match_images(image1, image2, model, device, amp, transforms):

    """
    Match given two images with each other using LoFTR model

    Parameters
    ----------
    image1: numpy.ndarray of shape (3, height, width)
        Batch of first images tensor

    image2: numpy.ndarray of shape (3, height, width)
        Batch of second images tensor

    model: torch.nn.Module
        LoFTR Model

    device: torch.device
        Location of the image1, image2 and the model

    amp: bool
        Whether to use auto mixed precision or not

    transforms: dict
        Dictionary of transform parameters

    Returns
    -------
    outputs: dict
        Model outputs
    """

    image1_raw_height, image1_raw_width = image1.shape[:2]
    image1 = get_image_tensor(
        image_path_or_array=image1,
        resize_shape=transforms['resize_shape'],
        resize_longest_edge=transforms['resize_longest_edge'],
        scale=transforms['scale'],
        grayscale=transforms['grayscale']
    )
    image1 = image1.to(device)
    image1_transformed_height, image1_transformed_width = image1.shape[2:]

    image2_raw_height, image2_raw_width = image2.shape[:2]
    image2 = get_image_tensor(
        image_path_or_array=image2,
        resize_shape=transforms['resize_shape'],
        resize_longest_edge=transforms['resize_longest_edge'],
        scale=transforms['scale'],
        grayscale=transforms['grayscale']
    )
    image2 = image2.to(device)
    image2_transformed_height, image2_transformed_width = image2.shape[2:]

    inputs = {
        'image0': image1,
        'image1': image2
    }

    with torch.no_grad():
        if amp:
            with torch.autocast(device_type=device.type, dtype=torch.float16):
                outputs = model(inputs)
        else:
            outputs = model(inputs)

    outputs = {
        'keypoints0': outputs['keypoints0'].detach().cpu().numpy(),
        'keypoints1': outputs['keypoints1'].detach().cpu().numpy(),
        'confidence': outputs['confidence'].detach().cpu().numpy(),
        'batch_indexes': outputs['batch_indexes'].detach().cpu().numpy()
    }

    outputs['keypoints0'][:, 0] *= image1_raw_width / image1_transformed_width
    outputs['keypoints0'][:, 1] *= image1_raw_height / image1_transformed_height
    outputs['keypoints1'][:, 0] *= image2_raw_width / image2_transformed_width
    outputs['keypoints1'][:, 1] *= image2_raw_height / image2_transformed_height

    return outputs


## 8. SuperGlue

In [12]:
def superglue_match_images(image1, image2, model, device, amp, transforms):

    """
    Match given two images with each other using SuperGlue model

    Parameters
    ----------
    image1: numpy.ndarray of shape (3, height, width)
        Batch of first images tensor

    image2: numpy.ndarray of shape (3, height, width)
        Batch of second images tensor

    model: torch.nn.Module
        SuperGlue Model

    device: torch.device
        Location of the image1, image2 and the model

    amp: bool
        Whether to use auto mixed precision or not

    transforms: dict
        Dictionary of transform parameters

    Returns
    -------
    outputs: dict
        Model outputs
    """

    image1_raw_height, image1_raw_width = image1.shape[:2]
    image1 = get_image_tensor(
        image_path_or_array=image1,
        resize=transforms['resize'],
        resize_shape=transforms['resize_shape'],
        resize_longest_edge=transforms['resize_longest_edge'],
        scale=transforms['scale'],
        grayscale=transforms['grayscale']
    )
    image1 = image1.to(device)
    image1_transformed_height, image1_transformed_width = image1.shape[2:]

    image2_raw_height, image2_raw_width = image2.shape[:2]
    image2 = get_image_tensor(
        image_path_or_array=image2,
        resize=transforms['resize'],
        resize_shape=transforms['resize_shape'],
        resize_longest_edge=transforms['resize_longest_edge'],
        scale=transforms['scale'],
        grayscale=transforms['grayscale']
    )
    image2 = image2.to(device)
    image2_transformed_height, image2_transformed_width = image2.shape[2:]

    inputs = {
        'image0': image1,
        'image1': image2
    }

    with torch.no_grad():
        if amp:
            with torch.autocast(device_type=device.type, dtype=torch.float16):
                outputs = model(inputs)
        else:
            outputs = model(inputs)

    outputs = {
        'keypoints0': outputs['keypoints0'][0].detach().cpu().numpy(),
        'scores0': outputs['scores0'][0].detach().cpu().numpy(),
        'descriptors0': outputs['descriptors0'][0].detach().cpu().numpy().T,
        'keypoints1': outputs['keypoints1'][0].detach().cpu().numpy(),
        'scores1': outputs['scores1'][0].detach().cpu().numpy(),
        'descriptors1': outputs['descriptors1'][0].detach().cpu().numpy().T,
        'matches0': outputs['matches0'][0].detach().cpu().numpy(),
        'matches1': outputs['matches1'][0].detach().cpu().numpy(),
        'matching_scores0': outputs['matching_scores0'][0].detach().cpu().numpy(),
        'matching_scores1': outputs['matching_scores1'][0].detach().cpu().numpy(),
    }

    matches_mask = outputs['matches0'] > -1

    for k in ['keypoints1', 'scores1', 'descriptors1', 'matches1', 'matching_scores1']:
        outputs[k] = outputs[k][outputs['matches0'][matches_mask]]

    for k in ['keypoints0', 'scores0', 'descriptors0', 'matches0', 'matching_scores0']:
        outputs[k] = outputs[k][matches_mask]

    outputs['keypoints0'][:, 0] *= image1_raw_width / image1_transformed_width
    outputs['keypoints0'][:, 1] *= image1_raw_height / image1_transformed_height
    outputs['keypoints1'][:, 0] *= image2_raw_width / image2_transformed_width
    outputs['keypoints1'][:, 1] *= image2_raw_height / image2_transformed_height

    return outputs


## 9. Inference

In [13]:
%%writefile ./config.yaml

dataset:
  haiper:
    bike:
      image_directory: images
    chairs:
      image_directory: images
    fountain:
      image_directory: images
  heritage:
    dioscuri:
      image_directory: images
    cyprus:
      image_directory: images
    wall:
      image_directory: images
  urban:
   kyiv-puppet-theater:
     image_directory: images

image_selection:
  image_count: 200
  criteria: step_size
  model:
    model_name: convnext_base
    pretrained: True
    model_args: {}
  transforms:
    resize_height: 512
    resize_width: 512
    normalize_mean: [0.485, 0.456, 0.406]
    normalize_std: [0.229, 0.224, 0.225]
    normalize_max_pixel_value: 255
  pooling_type: avg
  amp: False
  batch_size: 16
  num_workers: 2
  device: cuda

loftr:
  pretrained: null
  pretrained_weights_path: /kaggle/input/image-matching-challenge-2023-dataset/models/loftr/loftr_outdoor.ckpt
  transforms:
    resize_shape: 840
    resize_longest_edge: True
    scale: True
    grayscale: True

superglue:
  superpoint:
    descriptor_dim: 256
    nms_radius: 4
    keypoint_threshold: 0.01
    max_keypoints: -1
    remove_borders: 4
  superglue:
    descriptor_dim: 256
    weights: outdoor
    keypoint_encoder: [32, 64, 128, 256]
    sinkhorn_iterations: 100
    match_threshold: 0.2
  transforms:
    resize: False
    resize_shape: 1920
    resize_longest_edge: True
    scale: True
    grayscale: True

image_matching:
  amp: False
  batch_size: 1
  num_workers: 2
  device: cuda

colmap:
  device: cpu

sift_extraction:
  num_threads: -1
  max_image_size: 1400
  max_num_features: 8192
  first_octave: -1
  num_octaves: 4
  octave_resolution: 3
  peak_threshold: 0.0066
  edge_threshold: 10
  estimate_affine_shape: False
  max_num_orientations: 2
  upright: False
  darkness_adaptivity: False
  domain_size_pooling: False
  dsp_min_scale: 0.16
  dsp_max_scale: 3
  dsp_num_scales: 10
  normalization: 'L2'

sift_matching:
  num_threads: -1
  max_ratio: 0.9
  max_distance: 0.7
  cross_check: True
  max_num_matches: 32768
  max_error: 1.0
  confidence: 0.9
  min_num_trials: 100
  max_num_trials: 10000
  min_inlier_ratio: 0.25
  min_num_inliers: 15
  multiple_models: False
  guided_matching: False
  planar_scene: False

exhaustive_matching:
  block_size: 50

incremental_mapper:
  min_num_matches: 2
  ignore_watermarks: False
  multiple_models: True
  max_num_models: 50
  max_model_overlap: 20
  min_model_size: 10
  init_image_id1: -1
  init_image_id2: -1
  init_num_trials: 200
  extract_colors: False
  num_threads: -1
  min_focal_length_ratio: 0.1
  max_focal_length_ratio: 10.0
  max_extra_param: 1.0
  ba_refine_focal_length: True
  ba_refine_principal_point: False
  ba_refine_extra_params: True
  ba_min_num_residuals_for_multi_threading: 50000
  ba_local_num_images: 6
  ba_local_function_tolerance: 0.0
  ba_local_max_num_iterations: 25
  ba_global_use_pba: False
  ba_global_pba_gpu_index: -1
  ba_global_images_ratio: 1.1
  ba_global_points_ratio: 1.1
  ba_global_images_freq: 500
  ba_global_points_freq: 250000
  ba_global_function_tolerance: 0.0
  ba_global_max_num_iterations: 50
  ba_local_max_refinements: 2
  ba_local_max_refinement_change: 0.001
  ba_global_max_refinements: 5
  ba_global_max_refinement_change: 0.0005
  fix_existing_images: False

persistence:
  root_directory: inference

Writing ./config.yaml


In [14]:
config = yaml.load(open('config.yaml', 'r'), Loader=yaml.FullLoader)
config

{'dataset': {'haiper': {'bike': {'image_directory': 'images'},
   'chairs': {'image_directory': 'images'},
   'fountain': {'image_directory': 'images'}},
  'heritage': {'dioscuri': {'image_directory': 'images'},
   'cyprus': {'image_directory': 'images'},
   'wall': {'image_directory': 'images'}},
  'urban': {'kyiv-puppet-theater': {'image_directory': 'images'}}},
 'image_selection': {'image_count': 200,
  'criteria': 'step_size',
  'model': {'model_name': 'convnext_base',
   'pretrained': True,
   'model_args': {}},
  'transforms': {'resize_height': 512,
   'resize_width': 512,
   'normalize_mean': [0.485, 0.456, 0.406],
   'normalize_std': [0.229, 0.224, 0.225],
   'normalize_max_pixel_value': 255},
  'pooling_type': 'avg',
  'amp': False,
  'batch_size': 16,
  'num_workers': 2,
  'device': 'cuda'},
 'loftr': {'pretrained': None,
  'pretrained_weights_path': '/kaggle/input/image-matching-challenge-2023-dataset/models/loftr/loftr_outdoor.ckpt',
  'transforms': {'resize_shape': 840,
  

In [15]:
image_matching_device = torch.device(config['image_matching']['device'])

# Load LoFTR model with specified configurations
loftr_model = LoFTR(config['loftr']['pretrained'])
loftr_model.load_state_dict(torch.load(config['loftr']['pretrained_weights_path'])['state_dict'])
loftr_model = loftr_model.eval().to(image_matching_device)
loftr_transforms = config['loftr']['transforms']

# Load SuperPoint and SuperGlue model with specified configurations
superglue_model = Matching(config['superglue'])
superglue_model = superglue_model.eval().to(image_matching_device)
superglue_transforms = config['superglue']['transforms']

Loaded SuperPoint model
Loaded SuperGlue model ("outdoor" weights)


In [16]:
reconstruction_root_directory = Path('./inference')
reconstruction_root_directory.mkdir(parents=True, exist_ok=True)

datasets = [
    directory for directory in os.listdir(train_or_test_directory)
    if (train_or_test_directory / directory).is_dir() and (directory in df['dataset'].unique())
]

for dataset in datasets:
        
    dataset_directory = train_or_test_directory / dataset
    scenes = [
        directory for directory in os.listdir(dataset_directory)
        if (dataset_directory / directory).is_dir() and (directory in df['scene'].unique())
    ]
    
    for scene in scenes:
        
        scene_directory = dataset_directory / scene
        image_paths = sorted(glob(str(scene_directory / 'images' / '*')))
        scene_image_count = len(image_paths)
        
        scene_reconstruction_directory = reconstruction_root_directory / dataset / scene
        scene_reconstruction_directory.mkdir(parents=True, exist_ok=True)

        for file_or_directory in os.listdir(scene_reconstruction_directory):
            file_or_directory_path = scene_reconstruction_directory / file_or_directory
            if file_or_directory_path.is_file():
                os.remove(file_or_directory_path)
            elif file_or_directory_path.is_dir():
                shutil.rmtree(file_or_directory_path)

        # Create COLMAP database and its tables for the current reconstruction
        database_path = scene_reconstruction_directory / 'database.db'
        database_uri = f'file:{database_path}?mode=rwc'
        colmap_database = COLMAPDatabase.connect(database_uri, uri=True)
        colmap_database.create_tables()
        
        # Select images if scene image count is above the specified threshold
        if scene_image_count > config['image_selection']['image_count']:
            if config['image_selection']['criteria'] == 'similarity':
                # Load image selection model with specified configurations
                image_selection_device = torch.device(config['image_selection']['device'])
                image_selection_model = load_feature_extractor(**config['image_selection']['model'])
                image_selection_model = image_selection_model.eval().to(image_selection_device)
                image_selection_transforms = create_image_selection_transforms(**config['image_selection']['transforms'])

                image_selection_data_loader = prepare_dataloader(
                    image_paths=image_paths,
                    transforms=image_selection_transforms,
                    batch_size=config['image_selection']['batch_size'],
                    num_workers=config['image_selection']['num_workers']
                )
                image_selection_features = []

                for idx, inputs in enumerate(tqdm(image_selection_data_loader)):
                    inputs = inputs.to(image_selection_device)
                    batch_image_selection_features = extract_features(
                        inputs=inputs,
                        model=image_selection_model,
                        pooling_type=config['image_selection']['pooling_type'],
                        device=image_selection_device,
                        amp=config['image_selection']['amp']
                    )
                    image_selection_features.append(batch_image_selection_features)

                image_selection_features = torch.cat(image_selection_features, dim=0).numpy()
                # Select images with the highest mean cosine similarity because they are more likely to be registered
                image_paths = select_images(
                    image_paths=image_paths,
                    image_selection_features=image_selection_features,
                    image_count=config['image_selection']['image_count']
                )
                del image_selection_device, image_selection_model, image_selection_transforms, image_selection_data_loader, image_selection_features
            elif config['image_selection']['criteria'] == 'step_size':
                # Select images with step size
                step_size = int(np.ceil(len(image_paths) / config['image_selection']['image_count']))
                image_paths = image_paths[::step_size]
            else:
                raise ValueError(f'Invalid image selection criteria {config["image_selection"]["criteria"]}')
                
        scene_max_image_size = df.loc[df['scene'] == scene, ['image_height', 'image_width']].max().max()

        # Create brute force image pairs from image paths
        image_pair_indices = create_image_pairs(image_paths=image_paths)
        first_image_keypoints = []
        second_image_keypoints = []

        for image_pair_idx, (first_image_idx, second_image_idx) in enumerate(tqdm(image_pair_indices)):

            image1 = cv2.imread(str(image_paths[image_pair_indices[image_pair_idx][0]]))
            image1 = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)

            image2 = cv2.imread(str(image_paths[image_pair_indices[image_pair_idx][1]]))
            image2 = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)

            if scene_max_image_size > 3000:

                superglue_outputs = superglue_match_images(
                    image1=image1,
                    image2=image2,
                    model=superglue_model,
                    device=image_matching_device,
                    amp=False,
                    transforms={
                        'resize': True,
                        'resize_shape': 1920,
                        'resize_longest_edge': True,
                        'scale': True,
                        'grayscale': True,
                    }
                )

            else:

                superglue_outputs = superglue_match_images(
                    image1=image1,
                    image2=image2,
                    model=superglue_model,
                    device=image_matching_device,
                    amp=False,
                    transforms=superglue_transforms
                )

            first_image_keypoints.append(superglue_outputs['keypoints0'])
            second_image_keypoints.append(superglue_outputs['keypoints1'])

        write_matches(
            image_paths=image_paths,
            image_pair_indices=image_pair_indices,
            first_image_keypoints=first_image_keypoints,
            second_image_keypoints=second_image_keypoints,
            output_directory=scene_reconstruction_directory
        )

        push_to_database(
            colmap_database=colmap_database,
            dataset_directory=scene_reconstruction_directory,
            image_directory=scene_directory / 'images',
            camera_model='simple-radial',
            single_camera=True
        )
        
        sift_matching_options = pycolmap.SiftMatchingOptions(**config['sift_matching'])
        exhaustive_matching_options = pycolmap.ExhaustiveMatchingOptions(**config['exhaustive_matching'])

        pycolmap.match_exhaustive(
            database_path=database_path,
            sift_options=sift_matching_options,
            matching_options=exhaustive_matching_options,
            device=pycolmap.Device(config['colmap']['device']),
            verbose=True
        )

        incremental_mapper_options = pycolmap.IncrementalMapperOptions(**config['incremental_mapper'])

        reconstructions = pycolmap.incremental_mapping(
            database_path=database_path,
            image_path=scene_directory / 'images',
            output_path=scene_reconstruction_directory,
            options=incremental_mapper_options
        )
        
        if len(reconstructions) > 0:

            best_registered_image_count = 0
            best_reconstruction_idx = None

            for reconstruction_idx in reconstructions.keys():
                if reconstructions[reconstruction_idx].num_reg_images() > best_registered_image_count:
                    best_reconstruction_idx = reconstruction_idx
                    best_registered_image_count = reconstructions[reconstruction_idx].num_reg_images()

            best_reconstruction = reconstructions[best_reconstruction_idx]
        else:
            best_registered_image_count = 0
            best_reconstruction_idx = None
            best_reconstruction = None

        if verbose:
            print(
                f'''
                Dataset: {dataset} - Scene: {scene}
                Reconstruction count: {len(reconstructions)}
                Best reconstruction registered image count: {best_registered_image_count}
                '''
            )

        if best_reconstruction is not None:
            registered_images = {image.name: image for image in best_reconstruction.images.values()}
        else:
            registered_images = {}
            
        for idx, row in df.loc[(df['dataset'] == dataset) & (df['scene'] == scene)].iterrows():
            if row['image_id'] in registered_images:
                rotation_matrix_prediction = registered_images[row['image_id']].rotmat()
                translation_vector_prediction = registered_images[row['image_id']].tvec
                df.loc[idx, 'rotation_matrix'] = ';'.join([str(x) for x in rotation_matrix_prediction.reshape(-1)])
                df.loc[idx, 'translation_vector'] = ';'.join([str(x) for x in translation_vector_prediction.reshape(-1)])
            else:
                df.loc[idx, 'rotation_matrix'] = np.nan
                df.loc[idx, 'translation_vector'] = np.nan

        # Fill unregistered images rotation matrices with the prediction mean or zeros
        scene_rotation_matrix_predictions = df.loc[df['scene'] == scene, 'rotation_matrix'].dropna().apply(lambda x: np.array(str(x).split(';'), dtype=np.float64).reshape(1, 3, 3)).values
        if scene_rotation_matrix_predictions.shape[0] == 0:
            rotation_matrix_fill_value = np.zeros((3, 3))
        else:
            rotation_matrix_fill_value = np.mean(np.concatenate(scene_rotation_matrix_predictions, axis=0), axis=0)
        df.loc[(df['scene'] == scene) & (df['rotation_matrix'].isnull()), 'rotation_matrix'] = ';'.join([str(x) for x in rotation_matrix_fill_value.reshape(-1)])

        # Fill unregistered images translation vectors with the prediction mean or zeros
        scene_translation_vector_predictions = df.loc[df['scene'] == scene, 'translation_vector'].dropna().apply(lambda x: np.array(str(x).split(';'), dtype=np.float64).reshape(1, 3)).values
        if scene_translation_vector_predictions.shape[0] == 0:
            translation_vector_fill_value = np.zeros((3, 1))
        else:
            translation_vector_fill_value = np.mean(np.concatenate(scene_translation_vector_predictions, axis=0), axis=0)
        df.loc[(df['scene'] == scene) & (df['translation_vector'].isnull()), 'translation_vector'] = ';'.join([str(x) for x in translation_vector_fill_value.reshape(-1)])


100%|██████████| 15/15 [00:00<00:00, 1216.12it/s]



Exhaustive feature matching

Matching block [1/1, 1/1] in 9.871s
Elapsed time: 0.165 [minutes]

Loading database

Loading cameras... 1 in 0.000s
Loading matches... 45 in 0.000s
Loading images... 15 in 0.002s (connected 15)
Building correspondence graph... in 0.002s (ignored 0)

Elapsed time: 0.000 [minutes]


Finding good initial image pair


Initializing with image pair #7 and #1


Global bundle adjustment

iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_radius  ls_iter  iter_time  total_time
   0  5.438185e+03    0.00e+00    1.72e+05   0.00e+00   0.00e+00  1.00e+04        0    3.49e-03    1.33e-02
   1  3.298348e+03    2.14e+03    6.14e+05   1.45e+01   8.35e-01  1.43e+04        1    6.50e-03    1.98e-02
   2  2.951298e+03    3.47e+02    1.37e+05   3.04e+01   7.44e-01  1.62e+04        1    3.16e-03    2.30e-02
   3  2.940719e+03    1.06e+01    1.92e+05   3.37e+01   4.80e-02  9.32e+03        1    3.11e-03    2.61e-02
   4  2.693774e+03    2.47e+02    7.27e+04   1.

## 10. Submission

In [17]:
df_submission = df.loc[:, ['image_path', 'dataset', 'scene', 'rotation_matrix', 'translation_vector']]
df_submission.to_csv('submission.csv', index=False)

In [18]:
df_submission

,image_path,dataset,scene,rotation_matrix,translation_vector
0,haiper/bike/images/image_004.jpeg,haiper,bike,0.812080225964599;-0.1825040479125909;0.554272...,-1.1095910515919052;-1.9969069555757961;3.4243...
1,haiper/bike/images/image_029.jpeg,haiper,bike,0.369592223922397;-0.2867559565002921;0.883839...,-1.4564524899631375;-2.6847671925134544;5.3244...
2,haiper/bike/images/image_038.jpeg,haiper,bike,0.14455187093106092;-0.3645288182522968;0.9199...,-1.3074796374742483;-1.9726652982643855;3.4985...
3,haiper/bike/images/image_049.jpeg,haiper,bike,-0.4603243832999029;-0.23564902521201422;0.855...,0.09043729771431355;-2.691694512489184;4.25853...
4,haiper/bike/images/image_062.jpeg,haiper,bike,-0.881387717212236;0.2507332221145657;-0.40036...,1.4413653948995497;-1.9054030681489025;3.86473...
5,haiper/bike/images/image_076.jpeg,haiper,bike,0.575339778946482;0.3302348075213795;-0.748284...,1.0145536103240511;-1.1454807789193793;2.02291...
6,haiper/bike/images/image_088.jpeg,haiper,bike,0.9998804098557524;-4.959447925265357e-05;0.01...,-1.3749439190528743;-1.7921468419983213;2.6520...
7,haiper/bike/images/image_094.jpeg,haiper,bike,0.9891850321058375;-0.08647223607253951;0.1184...,-1.3209920598657354;-2.923572528560199;4.36381...
8,haiper/bike/images/image_101.jpeg,haiper,bike,0.9492955556094487;0.1501502199166534;-0.27621...,-0.8702004894405444;-0.3342177873043161;1.7002...
9,haiper/bike/images/image_115.jpeg,haiper,bike,0.34102840661975153;0.18152531869784239;-0.922...,0.9069415594501745;-2.3842047219071243;2.87325...
